In [ ]:
import json
import math
import pytz
import voltoolbox
import forward_fit

import datetime as dt
import numpy as np

from voltoolbox.fit.option_quotes import OptionSnapshot
from voltoolbox.calendar import nyse_calendar
from voltoolbox import BusinessTimeMeasure,bs_implied_volatility
from forward_fit import fit_forward_curve, act365_time, prepare_quotes_for_fit, OptionKernelRegression


file = 'vol_SPX_20210527_1857.json'
with open(file, 'r') as f:
    quotes_dict = json.loads(f.read())

quotes = OptionSnapshot.from_json_dict(quotes_dict)

fitted_forwards = fit_forward_curve(quotes)

pricing_dt = quotes.time_stamp
if pricing_dt.tzinfo is None:
    pricing_dt = pytz.UTC.localize(pricing_dt)  

business_time = BusinessTimeMeasure(nyse_calendar(), 0.5, 252.0)

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = np.array([6.0, 4.0]) * 2

slice_index_w = widgets.IntSlider(min=0, max =len(quotes.slices)-1)
@widgets.interact(slice_index=slice_index_w)
def plot_callput_parity(slice_index):      
    opt_sl = quotes.slices[slice_index]
    
    if not opt_sl.expiry in fitted_forwards:
        return
    
    t = business_time.distance(pricing_dt, opt_sl.expiry)
    forward = fitted_forwards[opt_sl.expiry]
    box_spd = 0.5 / 100.0
    discount = opt_sl.discount * math.exp(-box_spd * t)  

    put = opt_sl.put
    put_bids = np.array(put.bids) / discount
    vol_put_bids = np.array([max(0.0, bs_implied_volatility(forward, k, p, t, -1.0))
                             for k, p in zip(put.strikes, put_bids)])
    put_asks = np.array(put.asks) / discount
    vol_put_asks = np.array([max(0.0, bs_implied_volatility(forward, k, p, t, -1.0))
                             for k, p in zip(put.strikes, put_asks)])

    call = opt_sl.call
    call_bids = np.array(call.bids) / discount
    vol_call_bids = np.array([max(0.0, bs_implied_volatility(forward, k, p, t, 1.0))
                              for k, p in zip(call.strikes, call_bids)])
    call_asks = np.array(call.asks) / discount
    vol_call_asks = np.array([max(0.0, bs_implied_volatility(forward, k, p, t, 1.0))
                              for k, p in zip(call.strikes, call_asks)])

    plt.plot(put.strikes, vol_put_bids, marker='^', linestyle='none', color='blue')
    plt.plot(put.strikes, vol_put_asks, marker='v', linestyle='none', color='blue')
    plt.plot(call.strikes, vol_call_bids, marker='^', linestyle='none', color='red')
    plt.plot(call.strikes, vol_call_asks, marker='v', linestyle='none', color='red')
    
    plt.axvline(x=forward, color = 'black')
    plt.ylim((0, 1.1))
    plt.title(f'{opt_sl.symbol}  {opt_sl.expiry.date()}    fwd={round(forward, 2)}')
    plt.grid()

In [ ]:
plt.plot(fitted_forwards.keys(), fitted_forwards.values(), marker='+')
plt.grid()